# Laboratorio 1: Regresión en Boston

En este laboratorio deben hacer experimentos de regresión con el conjunto de datos "Boston house prices dataset".

Estudiarán el dataset, harán visualizaciones y seleccionarán atributos relevantes a mano.

Luego, entrenarán y evaluarán diferentes tipos de regresiones, buscando las configuraciones que mejores resultados den.

## Importaciones

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error

from rich.console import Console
from rich.table import Table

%matplotlib inline
# %matplotlib notebook

## Carga del Conjunto de Datos

Cargamos el conjunto de datos y vemos su contenido.

In [ ]:
boston = load_boston()
print(boston.DESCR)

## Ejercicio 1: Descripción de los Datos y la Tarea

Responda las siguientes preguntas:

1. ¿De qué se trata el conjunto de datos?
2. ¿Cuál es la variable objetivo que hay que predecir? ¿Qué significado tiene?
3. ¿Qué información (atributos) hay disponibles para hacer la predicción?
4. ¿Qué atributos imagina ud. que serán los más determinantes para la predicción?
5. ¿Qué problemas observa a priori en el conjunto de datos? ¿Observa posibles sesgos, riesgos, dilemas éticos, etc? Piense que los datos pueden ser utilizados para hacer predicciones futuras.

**No hace falta escribir código para responder estas preguntas.**

**1.** El conjunto de datos, tal como consta en la [descripción de los mismos](#Carga-del-Conjunto-de-Datos), se trata de valor de propiedades en Boston en base a 13 variables. Estos datos están tomados de un paper de Harrison, D. and Rubinfeld, D.L.: [Hedonic prices and the demand for clean air](https://www.researchgate.net/publication/4974606_Hedonic_housing_prices_and_the_demand_for_clean_air).

**2.** La variable objetivo es **MEDV, el valor medio (en miles de dólares) de una propiedad en Boston**. Lo que se debe tener en cuenta es que el dataset es del año 1978, por lo que si no se tienen en cuenta los incrementos por inflación, cualquier algoritmo de regresión que se haga con base en estos datos, la prediciión de valor de las viviendas serán para el año de publicación de paper, valor desactualizado para la presente época.

**3.** Se dispone de 13 variables según se explica en la [descripción del dataset](#Carga-del-Conjunto-de-Datos).

- CRIM: Tasa de criminalidad.
- ZN: Proporción de suelo residencial zonificado para lotes de más de 25,000 pies cuadrados.
- INDUS: Proporción de acres comerciales no minoristas por ciudad.
- CHAS: Variable ficticia de Charles River (= 1 si la casa limita con el ría; 0 en caso contrario).
- NOX: Concentración de óxidos nítricos (partes por 10 millones), es un indicio del nivel de contaminación.
- RM: Promedio de habitaciones.
- AGE: Proporción de unidades ocupadas por sus propietarios construidas antes de 1940.
- DIS: Distancias ponderadas a 5 zonas de trabajo en Boston.
- RAD: Índice de accesibilidad a autopistas.
- TAX: Tasa de impuesto a la propiedad de valor total por 10,000 dólares.
- PTRATIO: Realción entre cantidad de alumnos y maestros.
- B: Atributo que da un indicio de la población de personas de color (específicamente negros). B = 0 (63% de población negra). B aumenta en la medida en que disminuye la población negra.
- LSTAT: Porcentaje de población de bajo estatus

**4.** A priori, intuimos que el atributo más determinante para la predicción, es la cantidad de habitaciones (RM) que tiene la propiedad.

**5.** Este dataset presenta problemas éticos; de hecho, consta en la misma documentación de scikit.learn, a la que se puede acceder haciendo [click aquí](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html#sklearn-datasets-load-boston). El dataset es obsoleto en la versión 1.0 y ya no estará disponible en la versión 1.2.

Seguramente la razón de esto son los sesgos raciales implícitos en las variables LSTAT (porcentaje de población de bajo estatus social) y B (que depende de la proporción de gente negra).

Estos datos presentan sesgos que ponen en evidencia ciertos paradigmas sociales que, con justa razón, se pueden entender como discriminatorios. Por ende, pretender hacer predicciones con este dataset podría ofender a algunas personas o exponernos a acusaciones por discriminación y racismo.

## Ejercicio 2: Visualización de los Datos

1. Para cada atributo de entrada, haga una gráfica que muestre su relación con la variable objetivo.
2. Estudie las gráficas, identificando **a ojo** los atributos que a su criterio sean los más informativos para la predicción.
3. Para ud., ¿cuáles son esos atributos? Lístelos en orden de importancia.

A modo de preparación de los datos, creamos un DataFrame de Pandas con la información del dataset para usarlo en el ejercicio y el resto del Notebook.

In [ ]:
data = pd.DataFrame(data=boston.data, columns=boston.feature_names)
data['MEDV'] = boston.target
data.head()

**1.** En la próxima celda se encuentra una gráfica del valor medio de la propiedad en miles de dólares (MEDV) en función de cada uno de los atributos. 13 gráficos en total.

In [ ]:
for feature in boston['feature_names']:
    plt.figure()
    plt.scatter(data[feature], data.MEDV, facecolor="dodgerblue", edgecolor="k", label="datos")
    plt.xlabel(feature)
    plt.ylabel('MEDV')
    plt.show()

**2. y 3.** Después de estudiar "a ojo" cada una de las gráficas, se puede apreciar que los atributos que más influyen sobre el valor medio de la propiedad, ergo son los más informativos para la predicción, en orden de mayor a menor son:
- LSTAT: El valor de las propiedades disminuye notablemente en la medida en que se encuentra en zonas pobladas por personas de bajo estatus social.
- RM: El valor de la propiedad aumenta notablemente en función de la cantidad de habitaciones que posee.
- DIS: El valor de las propiedades aumenta en la medida en que se alejan de zonas que seguramente son industriales, por ser estas concentradores de empleo. Esta tendencia es coherente con NOX, que permite entender también que el valor de las propiedades disminuyen con el nivel de contaminación, que se encuentra muy relacionado con la industria.
- CRIM: Queda claro con este índice que las propiedades tienen poco valor en zonas de alta criminalidad.
- B: Este índice pone en evidencia que los valores de las propiedades son bajos en zonas con alta población de personas de color.

## Ejercicio 3: Regresión Lineal

1. Seleccione **un solo atributo** que considere puede ser el más apropiado.
2. Instancie una regresión lineal de **scikit-learn**, y entrénela usando sólo el atributo seleccionado.
3. Evalúe, calculando error cuadrático medio para los conjuntos de entrenamiento y evaluación.
4. Grafique el modelo resultante, junto con los puntos de entrenamiento y evaluación.
5. Interprete el resultado, haciendo algún comentario sobre las cualidades del modelo obtenido.

**Observación:** Con algunos atributos se puede obtener un error en test menor a 50.

**1.** Seleccionamos el atributo RM (cantidad de habitaciones), en virtud de que, entendemos, tiene mayor influencia sobre el valor de la propiedad.

In [ ]:
feature = 'RM'
X = data[[feature]]
y = data.MEDV

**2.** Instanciación y entrenamiento del atributo seleccionado.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=0)

**3.** Evaluación mediante cálculo del error medio cuatrático.

In [ ]:
regr = LinearRegression()
regr.fit(X_train, y_train)

print(f'El error medio cuadrático para el conjunto de entrenamiento es: {round(regr.score(X_train, y_train), 3)}.')
print(f'El error medio cuadrático para el conjunto de evaluación es: {round(regr.score(X_test, y_test), 3)}.')

**4.** Gráfica del modelo resultante

In [ ]:
plt.figure()
plt.scatter(X_train, y_train, facecolor="dodgerblue", edgecolor="k", label="train")
plt.scatter(X_test, y_test, facecolor="white", edgecolor="k", label="test")
plt.plot(X, regr.predict(X), color="tomato", label="modelo")
plt.xlabel(feature)
plt.ylabel('MEDV')
plt.legend()
plt.show()

**5.** El modelo aparenta tener un buen comportamiento. Representa la realidad que expresan los datos, que el valor medio de la propiedad es directamente proporcional a la cantidad de habitaciones.

La mayor objeción que se puede hacer del mismo es la predicción de un valor nulo para una propiedad con 4 habitaciones.

## Ejercicio 4: Regresión Polinomial

En este ejercicio deben entrenar regresiones polinomiales de diferente complejidad, siempre usando **scikit-learn**.

Deben usar **el mismo atributo** seleccionado para el ejercicio anterior.

1. Para varios grados de polinomio, haga lo siguiente:
    1. Instancie y entrene una regresión polinomial.
    2. Prediga y calcule error en entrenamiento y evaluación. Imprima los valores.
    3. Guarde los errores en una lista.
2. Grafique las curvas de error en términos del grado del polinomio.
3. Interprete la curva, identificando el punto en que comienza a haber sobreajuste, si lo hay.
4. Seleccione el modelo que mejor funcione, y grafique el modelo conjuntamente con los puntos.
5. Interprete el resultado, haciendo algún comentario sobre las cualidades del modelo obtenido.

**Observación:** Con algunos atributos se pueden obtener errores en test menores a 40 e incluso a 35.

In [ ]:
# 1.

def instantiate_pol_regr (degree, X, y):
    '''Instancia y entrena una regresión polinomial de grado deegree.
    
    Parámetros:
    -----------
    deegre: Grado del polinomio
    X: Conjunto de atributos
    y: Conjunto de datos objetivo
    
    Salida:
    -------
    model: Objeto con el modelo de la regresión polinimial
    '''
    pf = PolynomialFeatures(degree)
    lr = LinearRegression(fit_intercept=False)  # el bias ya esta como feature
    model = make_pipeline(pf, lr)
    model.fit(X, y)
    return model

reg_errores = []
max_degree = 35

deegrees = range(2, max_degree)

titulo = "Errores del modelo."
columnas = ["Grado del polinomio",
            "Error de entrenamiento",
            "Error de evaluación"]

table = Table(title=titulo)

for columna in columnas:
    table.add_column(columna, justify="center", style="blue")
            
for degree in deegrees:
    model = instantiate_pol_regr(degree, X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_test)
    train_error = mean_squared_error(y_train, y_train_pred)
    val_error = mean_squared_error(y_test, y_val_pred)
    reg_errores.append([degree, round(train_error, 2), round(val_error, 2)])
    table.add_row(str(degree),
                  str(round(train_error, 2)),
                  str(round(val_error, 2)))
    
console = Console()
console.print(table)

In [ ]:
# 2. Graficar curvas de error acá.

degree = [x[0] for x in reg_errores]
train_error = [x[1] for x in reg_errores]
test_error = [x[2] for x in reg_errores]

plt.figure()
plt.plot(degree, train_error)
plt.plot(degree, test_error)
plt.show()

**3.** El sobreajuste comienza a partir del grado 19. A partir de ese grado aumenta el error de evaluación y entrenamiento.

In [ ]:
# 4. Reconstruir mejor modelo acá y graficar.

model = instantiate_pol_regr(19, X_train, y_train)

X = np.linspace(0, 9, 100)
y = model.predict(X.reshape(-1, 1))

plt.figure()
plt.scatter(data.RM, data.MEDV, facecolor="dodgerblue", edgecolor="k", label="data")
plt.plot(X, y, color="tomato", label="modelo")
plt.xlim([0, 9])
plt.ylim([0, 52])
plt.legend()
plt.show()

**5.** El modelo ajusta con bastante precisión los valores extremos, cantidades máxima y mínimas de habitaciones, pero ello no significa que sea un buen modelo, ya que presenta un comportamientos que no tiene mucho sentido, a saber:
- Un máximo local en 3.3 aproximadamente
- Caída abrupta del valor de la propiedad para más de 8.8 habitaciones

Esto último significa que el modelo no es útil para extrapolar valores de propiedades por sobre 9 habitaciones.

Aún así, presenta un comportamiento adecuado entre 4.6 y 8.4.

## Ejercicio 5: Regresión con más de un Atributo

En este ejercicio deben entrenar regresiones que toman más de un atributo de entrada.

1. Seleccione **dos o tres atributos** entre los más relevantes encontrados en el ejercicio 2.
2. Repita el ejercicio anterior, pero usando los atributos seleccionados. No hace falta graficar el modelo final.
3. Interprete el resultado y compare con los ejercicios anteriores. ¿Se obtuvieron mejores modelos? ¿Porqué?

In [ ]:
# 1. Resolver acá. Ayuda (con dos atributos):

atributos_relevantes = ['RM', 'LSTAT']
x = 
selector = (boston['feature_names'] == 'CRIM') | (boston['feature_names'] == 'ZN')
X_train_fs = X_train[:, selector]
X_test_fs = X_test[:, selector]
X_train_fs.shape, X_test_fs.shape

In [ ]:
# 2. Resolver acá.

## Más ejercicios (opcionales)

### Ejercicio 6: A Todo Feature

Entrene y evalúe regresiones pero utilizando todos los atributos de entrada (va a andar mucho más lento). Estudie los resultados.

### Ejercicio 7: Regularización

Entrene y evalúe regresiones con regularización "ridge". Deberá probar distintos valores de "alpha" (fuerza de la regularización). ¿Mejoran los resultados?
